In [ ]:
import os
PATH = os.getcwd()
print(PATH)

/content/drive/MyDrive/inhalation


In [ ]:
from binary import main as binary_main
from binary_smote import main as binary_smote_main

In [ ]:
#SMOTE 기법 사용하지 않음
binary_main()

usage: colab_kernel_launcher.py [-h] [--tg_num TG_NUM] [--inhale_type INHALE_TYPE] [--model MODEL]
                                [--metric METRIC] [--num_run NUM_RUN] [--splitseed SPLITSEED]
                                [--smoteseed SMOTESEED] [--neighbor NEIGHBOR]
                                [--threshold THRESHOLD]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-a39ac3f2-d252-432e-a05b-2a8d2cf741f9.json


tg403 vapour logistic


[11:58:09] Explicit valence for atom # 0 O, 4, is greater than permitted
100%|██████████| 100/100 [10:20<00:00,  6.21s/it]


best param: {'C': 40, 'penalty': 'l1', 'solver': 'liblinear'}           
validation result           
precision: 0.492(0.047)           
recall: 0.505(0.040)           
accuracy: 0.787(0.015)           
auc: 0.728(0.013)           
f1: 0.491(0.037)
test result           
best param: {'C': 40, 'penalty': 'l1', 'solver': 'liblinear'}           
precision: 0.458           
recall: 0.579           
accuracy: 0.769           
auc: 0.667           
f1: 0.512


In [ ]:
#SMOTE 기법 사용
binary_smote_main()

SMOTE tg403 vapour logistic


usage: colab_kernel_launcher.py [-h] [--tg_num TG_NUM] [--inhale_type INHALE_TYPE] [--model MODEL]
                                [--metric METRIC] [--num_run NUM_RUN] [--splitseed SPLITSEED]
                                [--smoteseed SMOTESEED] [--neighbor NEIGHBOR]
                                [--threshold THRESHOLD]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-a39ac3f2-d252-432e-a05b-2a8d2cf741f9.json
[12:08:30] Explicit valence for atom # 0 O, 4, is greater than permitted
100%|██████████| 100/100 [16:22<00:00,  9.82s/it]


best param: {'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}           
validation result           
precision: 0.425(0.032)           
recall: 0.513(0.034)           
accuracy: 0.755(0.017)           
auc: 0.704(0.016)           
f1: 0.460(0.030)
test result           
best param: {'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}           
precision: 0.522           
recall: 0.632           
accuracy: 0.802           
auc: 0.751           
f1: 0.571


In [ ]:
# Binary.py
import json
import warnings
import numpy as np
from tqdm import tqdm

from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
    f1_score
)

from module.argument import get_parser
from module.read_data import (
    load_data,
    multiclass2binary
)
from module.smiles2fing import smiles2fing
from module.get_model import (
    load_model,
    load_hyperparameter
)
from module.common import (
    data_split,
    binary_cross_validation,
    print_best_param
)
warnings.filterwarnings('ignore')

# **Binary.py**

`Binary.py`는 이진 분류 문제를 해결하기 위한 머신러닝 모델을 구현하는 모듈입니다.

## **주요 라이브러리 및 모듈**

- **json**: JSON 파일 처리를 위해 사용되며, 모델 성능 결과를 JSON 형식으로 저장하는 데 사용됩니다.
- **warnings**: 경고 메시지를 필터링하기 위해 사용되며, 불필요한 경고를 숨기는 `warnings.filterwarnings('ignore')` 코드가 포함되어 있습니다.
- **numpy**: 다차원 배열 및 수치 연산을 처리하는 라이브러리입니다.
- **tqdm**: 반복 작업의 진행 상황을 시각화하기 위해 사용됩니다.
- **sklearn.metrics**: 모델의 성능 평가 지표들을 제공합니다 (예: precision_score, recall_score, accuracy_score 등).

## **사용자 정의 모듈**
- 아래는 사전에 정의한 모듈에서 binary.py에 사용되는 함수들만을 정리한 결과입니다.

#### module.argument
- `get_parser`: 커맨드라인 인자를 파싱하는 함수로, 스크립트 실행 옵션 설정에 사용됩니다.

#### module.read_data
- `load_data`: 데이터 로드를 위한 함수입니다.
- `multiclass2binary`: 다중 클래스 데이터를 이진 클래스로 변환합니다.

#### module.get_model
- `load_model`: 머신러닝 모델을 로드합니다.
- `load_hyperparameter`: 모델의 하이퍼파라미터 조합을 로드합니다.

#### module.common
- `data_split`: 데이터를 훈련/테스트 세트로 분할합니다.
- `binary_cross_validation`: 이진 분류를 위한 교차 검증을 수행합니다.
- `print_best_param`: 최적의 하이퍼파라미터를 선택합니다.



In [ ]:
def main():
    # 1. 인자 파싱과 설정 출력
    parser = get_parser()
    try:
        args = parser.parse_args()
    except:
        args = parser.parse_args([])

    # get_parser와 parse_args 함수를 사용해 command line parameter를 parsing합니다.

    print('=================================')
    print('tg%s %s %s' % (args.tg_num, args.inhale_type, args.model))

    # 학습에 사용할 data(이 경우, tg숫자와 흡기 종류에 해당), model 등등을 출력해줍니다.


    # 2. data loading / preprocessing

    x, y = load_data(path = 'data', tg_num = args.tg_num, inhale_type = args.inhale_type)

    # module.read_data에 정의된 load_data 함수를 사용하여 data 폴더 내부에 위치한 데이터를 로딩합니다.
    # 이때, 위에서 설정한 tg 설정에 맞는 데이터를 로딩합니다.

    y = multiclass2binary(y, args.tg_num)

    # multiclass2binary 함수를 사용하여 다항 분류에서 이진분류 형식으로 변환을 진행해줍니다.

    x_train, x_test, y_train, y_test = data_split(x, y, args.splitseed)

    # module.common에 정의된 data_split 함수를 사용하여 데이터를 training set과 test set으로 분할합니다.

    # 3. hyperparameter 설정 / cross validation 수행

    params = load_hyperparameter(args.model)
    # module.get_model 내부에 정의된 각 모델 별 hyperparameter를 사용하여 파싱한 모델에 맞는 하이퍼 파라미터를 로딩합니다.
    result = {}
    result['model'] = {}
    result['precision'] = {}
    result['recall'] = {}
    result['f1'] = {}
    result['accuracy'] = {}
    result['auc'] = {}

    # 교차 검증을 수행한 이후 결과를 저장할 result dictionary를 생성합니다.
    # result dictionary에는 모델과 각 평가 지표들이 존재합니다.
    # result dictionary에 key값에 대응되는 value는 또다른 dictionary로 설정되어
    # 중첩 dictionary를 생성하고, 다양한 설정에 대해 모델의 성능을 저장할 수 있도록 해줍니다.

    # 모델의 교차검증 / 성능평가 수행
    for p in tqdm(range(len(params))):
    # params 리스트에 있는 모든 hyperparameter 조합에 대해서 검증과 성능평가를 수행합니다.
    # tqdm은 진행률을 시각적으로 보여줍니다.
        result['model']['model'+str(p)] = params[p]
        result['precision']['model'+str(p)] = []
        result['recall']['model'+str(p)] = []
        result['f1']['model'+str(p)] = []
        result['accuracy']['model'+str(p)] = []
        result['auc']['model'+str(p)] = []
        # result dictionary에 성능 지표 리스트를 초기화해주고, 각 모델에 대한 정보로 result['model']을 초기화합니다.

        # parsing한 num_run번만큼 교차 검증을 진행합니다.
        for seed in range(args.num_run):
            model = load_model(model = args.model, seed = seed, param = params[p])
            # module.read_data에 저장된 load_model 함수를 통해 저장된 하이퍼파라미터, seed값 등을 활용하여 model 초기화를 진행합니다.
            cv_result = binary_cross_validation(model, x_train, y_train, seed)
            # module.common에 정의된 binary_cross_validation 함수를 사용하여 훈련 데이터에 대한 교차 검증을 수행합니다.

            result['precision']['model'+str(p)].append(cv_result['val_precision'])
            result['recall']['model'+str(p)].append(cv_result['val_recall'])
            result['f1']['model'+str(p)].append(cv_result['val_f1'])
            result['accuracy']['model'+str(p)].append(cv_result['val_accuracy'])
            result['auc']['model'+str(p)].append(cv_result['val_auc'])

            # binary_class_validation 결과로 생성된 성능 지표들의 값을 추출하여 result 딕셔너리의 각 키에 대응되는
            # 딕셔너리의 value로 설정된 list로 값을 append 시켜줍니다.

    # 4. 결과 저장 / 매개변수 선택

    json.dump(result, open(f'tg{args.tg_num}_val_results/binary/{args.inhale_type}_{args.model}.json', 'w'))

    # 교차 검증 결과를 JSON 파일로 저장합니다. 이는 현재 데이터 셋에 따라서 다양하게 구성될 수 있습니다.

    best_param = print_best_param(val_result = result, metric = args.metric)

    # parsing한 성능 지표와 이전 교차 검증 결과를 module.common에 정의된 print_best_param에 넘겨주어
    # 최적의 매개변수 조합을 선택하게 됩니다.

    m = list(result['model'].keys())[list(result['model'].values()).index(best_param)]

    # result dictionary에서 best_param에 맞는 모델의 키 값을 반환합니다.

    # val result
    precision = result['precision'][m]
    recall = result['recall'][m]
    acc = result['accuracy'][m]
    auc = result['auc'][m]
    f1 = result['f1'][m]

    # 선택된 모델의 성능지표를 추출합니다.

    print(f"best param: {best_param} \
          \nvalidation result \
          \nprecision: {np.mean(precision):.3f}({np.std(precision):.3f}) \
          \nrecall: {np.mean(recall):.3f}({np.std(recall):.3f}) \
          \naccuracy: {np.mean(acc):.3f}({np.std(acc):.3f}) \
          \nauc: {np.mean(auc):.3f}({np.std(auc):.3f}) \
          \nf1: {np.mean(f1):.3f}({np.std(f1):.3f})")

          # 선택된 모델의 지표 각각의 평균과 표준편차를 계산하여 출력해줍니다.

    # 5. Test set 모델 평가
    model = load_model(model = args.model, seed = seed, param = best_param)

    # module.read_data에 저장된 load_model 함수를 통해 training 과정에서 찾은 최적의
    # 하이퍼파라미터, seed값 등을 활용하여 model 초기화를 진행합니다.

    model.fit(x_train, y_train)

    # 모델 훈련을 진행합니다.

    if args.model == 'plsda':
        pred_score = model.predict(x_test)
        pred = np.where(pred_score < 0.5, 0, 1).reshape(-1)
        # 모델이 부분 최소 제곱법이 경우 예측 확률에 따라 이진 분류를 진행합니다.
    else:
        pred = model.predict(x_test)
        pred_score = model.predict_proba(x_test)[:, 1]
        # 나머지 모델의 경우 model.predict와 model.predict_proba 메소드를 사용하여
        # 예측값과 확률을 계산해줍니다.

    print(f'test result \
          \nbest param: {best_param} \
          \nprecision: {precision_score(y_test, pred):.3f} \
          \nrecall: {recall_score(y_test, pred):.3f} \
          \naccuracy: {accuracy_score(y_test, pred):.3f} \
          \nauc: {roc_auc_score(y_test, pred_score):.3f} \
          \nf1: {f1_score(y_test, pred):.3f}')

    # test model에 대해서 성능 지표들을 계산하여 출력해줍니다.
    # 이는 모델의 test set에 대한 성능 평가입니다.

if __name__ == '__main__':
    main()


# **binary.main()**

- **main** 함수는 머신러닝 생성 과정을 수행하기 위한 함수입니다.

# **Main 함수 동작 과정**
1. **인자 파싱 및 설정 출력**:
    - 커맨드라인 인자를 파싱하여 설정을 불러옵니다.
    - 파싱된 인자를 사용해 실험에 사용될 데이터셋(`tg_num`), 흡입 유형(`inhale_type`), 모델(`model`) 등을 출력합니다.

2. **데이터 로딩 및 전처리**:
    - `load_data`를 사용해 지정된 데이터를 로딩합니다.
    - `multiclass2binary`를 통해 데이터를 이진 분류 형식으로 변환합니다.
    - `data_split` 함수로 데이터를 훈련 세트와 테스트 세트로 나눕니다.

3. **하이퍼파라미터 설정 및 교차 검증**:
    - `load_hyperparameter`로 사전에 정의된 하이퍼파라미터를 불러옵니다.
    - `binary_cross_validation`으로 교차 검증을 수행해 모델 성능을 평가합니다.  
4. **결과 저장 및 최적 매개변수 선택**:
    - 교차 검증 결과를 JSON 파일로 저장합니다.
    - `print_best_param`을 통해 최적의 매개변수를 선택합니다.

5. **테스트 세트에서 모델 평가**:
    - 최적의 매개변수로 모델을 테스트 세트에 적용합니다.
    - 모델의 성능을 정밀도, 재현율, 정확도, AUC, F1 점수로 평가하고 결과를 출력합니다.

```python
# 자세한 동작 과정은 주석을 참조하세요.
```


In [ ]:
# Binary_smote.py
import json
import warnings

import numpy as np
from tqdm import tqdm
from imblearn.over_sampling import SMOTE

from sklearn.metrics import (
    precision_score,
    recall_score,
    accuracy_score,
    roc_auc_score,
    f1_score
)

from module.argument import get_parser
from module.read_data import (
    load_data,
    multiclass2binary
)
from module.smiles2fing import smiles2fing
from module.get_model import (
    load_model,
    load_hyperparameter
)
from module.common import (
    data_split,
    binary_smote_cross_validation,
    print_best_param
)

warnings.filterwarnings('ignore')

# **binary_smote.py**

`binary_smote.py`는 SMOTE 방식을 사용하여 불균형 데이터를 가진 이진 분류 문제를 해결하기 위한 머신러닝 모델을 구현하는 모듈입니다.

## **주요 라이브러리 및 모듈**

- **json**: JSON 파일 처리를 위해 사용되며, 모델 성능 결과를 JSON 형식으로 저장하는 데 사용됩니다.
- **warnings**: 경고 메시지를 필터링하기 위해 사용되며, 불필요한 경고를 숨기는 `warnings.filterwarnings('ignore')` 코드가 포함되어 있습니다.
- **numpy**: 다차원 배열 및 수치 연산을 처리하는 라이브러리입니다.
- **tqdm**: 반복 작업의 진행 상황을 시각화하기 위해 사용됩니다.
- **imblearn.over_sampling.SMOTE**: 불균형 데이터셋을 처리하기 위해 SMOTE 오버샘플링을 적용하기 위해 사용되는 라이브러리입니다.
- **sklearn.metrics**: 모델의 성능 평가 지표들을 제공합니다 (예: precision_score, recall_score, accuracy_score 등).

## **사용자 정의 모듈**
- 아래는 사전에 정의한 모듈에서 binary_smote.py에 사용되는 함수들만을 정리한 결과입니다.

#### module.argument
- `get_parser`: 커맨드라인 인자를 파싱하는 함수로, 스크립트 실행 옵션 설정에 사용됩니다.

#### module.read_data
- `load_data`: 데이터 로드를 위한 함수입니다.
- `multiclass2binary`: 다중 클래스 데이터를 이진 클래스로 변환합니다.

#### module.get_model
- `load_model`: 머신러닝 모델을 로드합니다.
- `load_hyperparameter`: 모델의 하이퍼파라미터 조합을 로드합니다.

#### module.common
- `data_split`: 데이터를 훈련/테스트 세트로 분할합니다.
- `binary_cross_validation`: 이진 분류를 위한 교차 검증을 수행합니다.
- `print_best_param`: 최적의 하이퍼파라미터를 선택합니다.


In [ ]:
def main():
    # 1. 인자 파싱과 설정 출력
    parser = get_parser()
    try:
        args = parser.parse_args()
    except:
        args = parser.parse_args([])
      # get_parser와 parse_args 함수를 사용해 command line parameter를 parsing합니다.
    print('=================================')
    print('SMOTE tg%s %s %s' % (args.tg_num, args.inhale_type, args.model))

    # 파싱된 인자를 사용해 실험에 사용될 데이터셋(tg_num), 흡입 유형(inhale_type), 모델(model) 등을 출력합니다.

    # 2. data loading / preprocessing

    x, y = load_data(path = 'data', tg_num = args.tg_num, inhale_type = args.inhale_type)
    y = multiclass2binary(y, args.tg_num)

    # module.read_data에 정의된 load_data 함수를 사용하여 data 폴더 내부에 위치한 데이터를 로딩합니다.
    # 이때, 위에서 설정한 tg 설정에 맞는 데이터를 로딩합니다.

    # multiclass2binary 함수를 사용하여 다항 분류에서 이진분류 형식으로 변환을 진행해줍니다.

    x_train, x_test, y_train, y_test = data_split(x, y, args.splitseed)

    # module.common에 정의된 data_split 함수를 사용하여 데이터를 training set과 test set으로 분할합니다.

    # 3. hyperparameter 설정 / cross validation 수행

    params = load_hyperparameter(args.model)

    # module.get_model 내부에 정의된 각 모델 별 hyperparameter를 사용하여 파싱한 모델에 맞는 하이퍼 파라미터를 로딩합니다.

    result = {}
    result['model'] = {}
    result['precision'] = {}
    result['recall'] = {}
    result['f1'] = {}
    result['accuracy'] = {}
    result['auc'] = {}

    # 교차 검증을 수행한 이후 결과를 저장할 result dictionary를 생성합니다.
    # result dictionary에는 모델과 각 평가 지표들이 존재합니다.
    # result dictionary에 key값에 대응되는 value는 또다른 dictionary로 설정되어
    # 중첩 dictionary를 생성하고, 다양한 설정에 대해 모델의 성능을 저장할 수 있도록 해줍니다.

    for p in tqdm(range(len(params))):
    # 모델의 교차검증 / 성능평가 수행
    # params 리스트에 있는 모든 hyperparameter 조합에 대해서 검증과 성능평가를 수행합니다.
    # tqdm은 진행률을 시각적으로 보여줍니다.
        result['model']['model'+str(p)] = params[p]
        result['precision']['model'+str(p)] = []
        result['recall']['model'+str(p)] = []
        result['f1']['model'+str(p)] = []
        result['accuracy']['model'+str(p)] = []
        result['auc']['model'+str(p)] = []
        # result dictionary에 성능 지표 리스트를 초기화해주고, 각 모델에 대한 정보로 result['model']을 초기합니다.

        # parsing한 num_run번만큼 교차 검증을 진행합니다.
        for seed in range(args.num_run):
            model = load_model(model = args.model, seed = seed, param = params[p])
            # module.read_data에 저장된 load_model 함수를 통해 저장된 하이퍼파라미터, seed값 등을 활용하여 model 초기화를 진행합니다.
            cv_result = binary_smote_cross_validation(model, x_train, y_train, seed, args)
            # module.common에 정의된 binary_smote_cross_validation 함수를 사용하여 SMOTE 기법을 활용한 교차 검증을 수행합니다.

            result['precision']['model'+str(p)].append(cv_result['val_precision'])
            result['recall']['model'+str(p)].append(cv_result['val_recall'])
            result['f1']['model'+str(p)].append(cv_result['val_f1'])
            result['accuracy']['model'+str(p)].append(cv_result['val_accuracy'])
            result['auc']['model'+str(p)].append(cv_result['val_auc'])

            # binary_smote_cross_validation 결과로 생성된 성능 지표들의 값을 추출하여 result 딕셔너리의 각 키에 대응되는
            # 딕셔너리의 value로 설정된 list에 값을 append 시켜줍니다.


    # 4. 결과 저장 / 매개변수 선택
    json.dump(result, open(f'tg{args.tg_num}_val_results/binary_smote{args.neighbor}/{args.inhale_type}_{args.model}.json', 'w'))

    # 교차 검증 결과를 JSON 파일로 저장합니다. 이는 현재 데이터 셋에 따라서 다양하게 구성될 수 있습니다.

    best_param = print_best_param(val_result = result, metric = args.metric)

    # parsing한 성능 지표와 이전 교차 검증 결과를 module.common에 정의된 print_best_param에 넘겨주어
    # 최적의 매개변수 조합을 선택하게 됩니다.

    m = list(result['model'].keys())[list(result['model'].values()).index(best_param)]

    # result dictionary에서 best_param에 맞는 모델의 키 값을 반환합니다.

    # val result
    precision = result['precision'][m]
    recall = result['recall'][m]
    acc = result['accuracy'][m]
    auc = result['auc'][m]
    f1 = result['f1'][m]

    # 선택된 모델의 성능지표를 추출합니다.

    print(f"best param: {best_param} \
          \nvalidation result \
          \nprecision: {np.mean(precision):.3f}({np.std(precision):.3f}) \
          \nrecall: {np.mean(recall):.3f}({np.std(recall):.3f}) \
          \naccuracy: {np.mean(acc):.3f}({np.std(acc):.3f}) \
          \nauc: {np.mean(auc):.3f}({np.std(auc):.3f}) \
          \nf1: {np.mean(f1):.3f}({np.std(f1):.3f})")

          # 선택된 모델의 지표 각각의 평균과 표준편차를 계산하여 출력해줍니다.


    # 5. Test set 모델 평가
    smote = SMOTE(random_state = args.smoteseed, k_neighbors = args.neighbor)
    # parsing한 정보를 바탕으로 작은 데이터셋을 오버샘플링하는 SMOTE 함수를 구현합니다.

    x_train, y_train = smote.fit_resample(x_train, y_train)

    # 생성한 smote 함수를 사용하여 오버샘플링을 진행합니다.

    model = load_model(model = args.model, seed = seed, param = best_param)

    # Training 과정에서 최적의 성능을 보인 모델과 파라미터를 시드와 함께 불러옵니다.

    model.fit(x_train, y_train)

    # 모델 훈련을 진행합니다.

    if args.model == 'plsda':
        pred_score = model.predict(x_test)
        pred = np.where(pred_score < 0.5, 0, 1).reshape(-1)
        # 모델이 부분 최소 제곱법이 경우 예측 확률에 따라 이진 분류를 진행합니다.
    else:
        pred = model.predict(x_test)
        pred_score = model.predict_proba(x_test)[:, 1]
        # 나머지 모델의 경우 model.predict와 model.predict_proba 메소드를 사용하여
        # 예측값과 확률을 계산해줍니다.

    print(f'test result \
          \nbest param: {best_param} \
          \nprecision: {precision_score(y_test, pred):.3f} \
          \nrecall: {recall_score(y_test, pred):.3f} \
          \naccuracy: {accuracy_score(y_test, pred):.3f} \
          \nauc: {roc_auc_score(y_test, pred_score):.3f} \
          \nf1: {f1_score(y_test, pred):.3f}')
    # test model에 대해서 성능 지표들을 계산하여 출력해줍니다.
    # 이는 모델의 test set에 대한 성능 평가입니다.

if __name__ == '__main__':
    main()


# **binary.main()**

## **Main 함수 동작 과정**
1. **인자 파싱 및 설정 출력**:
    - 커맨드라인 인자를 파싱하여 실험 설정을 불러오고 출력합니다. 이는 데이터셋(`tg_num`), 흡입 유형(`inhale_type`), 모델(`model`) 등을 포함합니다.

2. **데이터 로딩 및 전처리**:
    - `load_data` 함수를 이용해 지정된 데이터를 로딩합니다.
    - `multiclass2binary`를 통해 데이터를 이진 분류 형식으로 변환합니다.
    - `data_split`을 사용해 데이터를 train / test set로 분할합니다.

3. **하이퍼파라미터 설정 및 교차 검증**:
    - `load_hyperparameter`를 사용해 사전에 정의된 하이퍼파라미터를 불러옵니다.
    - `binary_cross_validation`을 통해 교차 검증을 수행하고 모델 성능을 평가합니다.

4. **결과 저장 및 최적 매개변수 선택**:
    - 교차 검증 결과를 JSON 파일로 저장합니다.
    - `print_best_param`으로 최적의 매개변수를 선택합니다.

5. **test set에서 모델 평가**:
    - 최적의 매개변수와 SMOTE 방식을 사용해 모델을 훈련하고 test set에서 모델을 평가합니다.
    - 성능 지표로는 정밀도, 재현율, 정확도, AUC, F1 점수를 사용하여 결과를 출력합니다.

```python
# 자세한 동작 과정은 코드 내 주석을 참조하세요.
